# Install libraries

In [2]:
# install libraries
!pip install catboost


In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import seaborn as sns
import plotly.express as px
from IPython.display import display

# score metrics and splitting libraries
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_percentage_error, r2_score
from sklearn.feature_selection import SelectFromModel
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.pipeline import Pipeline

# ML algorithms from sklearn
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.preprocessing import PolynomialFeatures
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, VotingRegressor

# extreme algorithms
from xgboost import XGBRegressor
from catboost import CatBoostRegressor
from lightgbm import LGBMRegressor

# warning turn off
import warnings
warnings.filterwarnings('ignore')

In [2]:
#set display max column to see all column
pd.set_option("display.max_columns", None)
pd.get_option("display.max_columns")

#set display max column to see all column
pd.set_option("display.max_rows", 150)
pd.get_option("display.max_rows")

150

# Load Datasets

In [3]:
# load datasets

train = pd.read_csv('/kaggle/input/cleaned-delays-data/cleaned_train.csv')
test = pd.read_csv('/kaggle/input/cleaned-delays-data/cleaned_test.csv')

train.head()

,Weekday,Month_of_Year,Day_of_Month,Departure_State,Arrival_State,Flight_Cancelled,Departure_Delay,Arrival_Delay,Taxi_Out_Time,Taxi_In_Time,Flight_Diverted,Flight_Duration,Flight_Distance,Origin_Temperature,Destination_Temperature,Origin_Wind_Speed,Destination_Wind_Speed,Origin_Precipitation,Destination_Precipitation,Scheduled_Departure_Time_Minutes,Scheduled_Arrival_Time_Minutes,Actual_Departure_Time_Minutes
0,2,6,21,3,3,0,0.088687,-4.178483,16.0,5.0,0,23.0,110.0,13.279939,20.470690,18.045064,12.910265,0.0,0.0,736,784,736
1,3,1,5,22,50,0,13.027852,5.042185,34.0,5.0,0,36.0,119.0,17.816202,13.967273,21.606228,17.976362,0.0,0.1,1185,1255,1185
2,2,3,22,8,44,0,-1.802698,-0.206932,10.0,11.0,0,125.0,641.0,24.562566,14.509228,24.946489,22.630553,0.0,0.0,420,564,418
3,4,7,14,49,32,0,13.027852,14.006092,23.0,7.0,0,130.0,867.0,8.817992,10.866812,17.426336,17.401007,0.0,0.0,1290,1439,1293
4,1,5,30,4,4,0,-2.181755,-14.067374,10.0,5.0,0,58.0,417.0,9.360464,15.977111,19.001179,16.884960,0.0,0.0,470,555,468


We split the train, contains target variable, to train and validation to train and test models performance and accuracy.

In [4]:
# split train to train set and validation sets, use 75 - 25 % approach
# Shuffle the indices
indices = np.arange(len(train))
np.random.shuffle(indices)

# Define the split ratio
split_ratio = 0.75

# Calculate split indices
split_index = int(len(train) * split_ratio)

# Split the data
df_train = train.iloc[indices[:split_index], :]
df_test = train.iloc[indices[split_index:], :]

df_train.reset_index(drop = True, inplace = True)
df_test.reset_index(drop = True, inplace = True)

print(f'Train size : {df_train.shape}')
print(f'Test size : {df_test.shape}')

Train size : (1065015, 22)
Test size : (355006, 22)


In [5]:
display(df_train.head())

print('*' * 70)
df_test.head()

,Weekday,Month_of_Year,Day_of_Month,Departure_State,Arrival_State,Flight_Cancelled,Departure_Delay,Arrival_Delay,Taxi_Out_Time,Taxi_In_Time,Flight_Diverted,Flight_Duration,Flight_Distance,Origin_Temperature,Destination_Temperature,Origin_Wind_Speed,Destination_Wind_Speed,Origin_Precipitation,Destination_Precipitation,Scheduled_Departure_Time_Minutes,Scheduled_Arrival_Time_Minutes,Actual_Departure_Time_Minutes
0,3,6,1,34,13,0,-7.731546,-4.833842,16.0,20.0,0,66.00000,343.0,11.502819,17.527869,16.048030,17.450891,0.0,0.0,850,889,842
1,7,6,26,41,8,0,-7.012920,-21.974180,14.0,4.0,0,156.00000,1334.0,13.080554,17.609027,21.572901,22.012316,0.0,0.0,1174,1423,1167
2,4,2,17,9,14,0,-3.002681,-12.384460,7.0,4.0,0,66.00000,432.0,19.409724,6.334498,16.425320,17.252616,0.0,0.0,755,841,752
3,7,6,19,9,16,0,-4.267220,-8.864074,16.0,4.0,0,111.10592,304.0,13.460187,19.808485,20.077378,20.082186,0.0,0.0,1065,1141,1061
4,6,1,1,26,37,0,9.082084,-1.049840,14.0,8.0,0,111.10592,449.0,9.749476,12.148609,20.077378,20.082186,0.0,0.0,1107,1206,1116


**********************************************************************


,Weekday,Month_of_Year,Day_of_Month,Departure_State,Arrival_State,Flight_Cancelled,Departure_Delay,Arrival_Delay,Taxi_Out_Time,Taxi_In_Time,Flight_Diverted,Flight_Duration,Flight_Distance,Origin_Temperature,Destination_Temperature,Origin_Wind_Speed,Destination_Wind_Speed,Origin_Precipitation,Destination_Precipitation,Scheduled_Departure_Time_Minutes,Scheduled_Arrival_Time_Minutes,Actual_Departure_Time_Minutes
0,3,4,13,3,3,0,13.027852,10.136899,30.0,8.0,0,31.00000,799.232715,16.596518,16.598338,15.022597,19.300735,0.0,0.0,506,559,500
1,5,7,15,5,23,0,54.043945,51.228317,16.0,5.0,0,71.00000,533.000000,19.345465,14.326943,20.911186,12.574245,0.0,2.0,900,1055,954
2,1,3,28,39,8,0,18.972616,6.989461,13.0,21.0,0,111.10592,1072.000000,25.814961,16.933031,20.077378,20.082186,0.0,0.0,1145,1336,1164
3,2,5,17,33,5,0,-3.776253,-26.021951,11.0,7.0,0,236.00000,1620.000000,18.021252,20.072836,23.183333,23.890829,0.0,0.0,735,891,731
4,5,6,17,23,37,0,13.106595,-0.839466,12.0,3.0,0,101.00000,814.000000,18.315687,25.630093,22.600244,21.858314,0.0,0.0,645,835,658


# Parameter Tuning

When working with a large dataset like ours (1 million rows), finding the best parameters for regression models while ensuring efficient model training and evaluation cost to much computation and time.

To handle this issue we use **Sampling for Hyperparameter Tuning**. It means that instead of using the entire dataset for hyperparameter tuning, we can use a representative sample of data. This will significantly reduce computation time and allow us to perform thorough hyperparameter tuning.

In [6]:
# use 15% of the data for tuning
sample_size = int(0.15 * len(df_train))
df_sample = df_train.sample(n = sample_size, random_state = 42)

X_sample = df_sample.drop(columns=['Arrival_Delay'])
y_sample = df_sample['Arrival_Delay']

## Parameters for baseline models

In [16]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_sample, y_sample, test_size = 0.3, random_state = 0)

X_train.shape

(111826, 21)

In [7]:
from sklearn.metrics import accuracy_score, mean_absolute_percentage_error


# import packages for hyperparameters tuning
from hyperopt import STATUS_OK, Trials, fmin, hp, tpe


In [46]:
# Define the parameter space
space = {
    'max_depth': hp.quniform("max_depth", 3, 18, 1),
    'gamma': hp.uniform('gamma', 1, 9),
    'subsample': hp.uniform('subsample', 0.1, 1.0), 
    'reg_alpha': hp.quniform('reg_alpha', 40, 180, 1),
    'colsample_bytree': hp.uniform('colsample_bytree', 0.5, 1.0),
    'min_child_weight': hp.quniform('min_child_weight', 0, 10, 1),
    'n_estimators': hp.quniform('n_estimators', 180, 2000, 50),
    'learning_rate': hp.uniform('learning_rate', 0.001, 1),
    'seed': 0
}


In [47]:
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error
from hyperopt import STATUS_OK

# Define the objective function
def objective(space):
    reg_xgb = XGBRegressor(
        n_estimators=int(space['n_estimators']),
        max_depth=int(space['max_depth']),
        gamma=space['gamma'],
        subsample=space['subsample'],
        reg_alpha=int(space['reg_alpha']),
        colsample_bytree=space['colsample_bytree'],
        min_child_weight=int(space['min_child_weight']),
        learning_rate=space['learning_rate'],
        seed=int(space['seed']),
        verbosity=0
    )
    
    evaluation = [(X_train, y_train), (X_test, y_test)]
    
    reg_xgb.fit(X_train, y_train,
            eval_set=evaluation, eval_metric="rmse",
            early_stopping_rounds=10, verbose=False)
    
    pred = reg_xgb.predict(X_test)
    mape = mean_absolute_percentage_error(y_test, pred)
    
    print("MAPE:", mape)
    
    return {'loss': mape, 'status': STATUS_OK}


In [49]:
%%time

trials = Trials()

best_hyperparams = fmin(fn = objective,
                        space = space,
                        algo = tpe.suggest,
                        max_evals = 100,
                        trials = trials)

MAPE:                                                  
7.397385615584983                                      
MAPE:                                                                           
6.726652419498571                                                               
MAPE:                                                                           
6.508633178664373                                                               
MAPE:                                                                           
7.588509278814695                                                               
MAPE:                                                                           
8.16048401451062                                                                
MAPE:                                                                           
5.960375890274773                                                               
MAPE:                                                                         

In [51]:
print("The best hyperparameters are : ","\n")
print(best_hyperparams)

The best hyperparameters are :  

{'colsample_bytree': 0.9698294480528181, 'gamma': 3.7604998424073166, 'learning_rate': 0.2718006286831415, 'max_depth': 8.0, 'min_child_weight': 1.0, 'n_estimators': 1000.0, 'reg_alpha': 146.0, 'subsample': 0.7423469165690778}


In [20]:
best_parameters = {'colsample_bytree': 0.9698294480528181,
 'gamma': 3.7604998424073166,
 'learning_rate': 0.2718006286831415,
 'max_depth': 8,
 'min_child_weight': 1,
 'n_estimators': 983,
 'reg_alpha': 146,
 'subsample': 0.7423469165690778,
 'random_state': 123}

best_parameters

{'colsample_bytree': 0.9698294480528181,
 'gamma': 3.7604998424073166,
 'learning_rate': 0.2718006286831415,
 'max_depth': 8,
 'min_child_weight': 1,
 'n_estimators': 983,
 'reg_alpha': 146,
 'subsample': 0.7423469165690778,
 'random_state': 123}

In [21]:
xgb_optuna_params = {
    'n_estimators': 929,
    'alpha': 2.287466581490129e-05,
    'subsample': 0.8766675651018592,
    'colsample_bytree': 0.288332829334817,
    'max_depth': 8,
    'min_child_weight': 6,
    'learning_rate': 0.024083411832750343,
    'gamma': 0.001816649055813574
}

xgb_optuna_params

{'n_estimators': 929,
 'alpha': 2.287466581490129e-05,
 'subsample': 0.8766675651018592,
 'colsample_bytree': 0.288332829334817,
 'max_depth': 8,
 'min_child_weight': 6,
 'learning_rate': 0.024083411832750343,
 'gamma': 0.001816649055813574}

In [11]:
# test model with using big data
X_train = df_train.drop(columns=['Arrival_Delay'])
X_test = df_test.drop(columns=['Arrival_Delay'])
y_train = df_train['Arrival_Delay']
y_test = df_test['Arrival_Delay']

print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(1065015, 21) (1065015,)
(355006, 21) (355006,)


### Without cross validation

In [37]:
from xgboost import XGBRegressor

evaluation = [(X_train, y_train), (X_test.sample(n = sample_size, random_state = 42), y_test.sample(n = sample_size, random_state = 42))]
    
for set in evaluation:
    print(set[0].shape, set[1].shape)


(1065015, 21) (1065015,)
(159752, 21) (159752,)


In [44]:
# Create the XGBRegressor model with the best parameters
xgb_reg = XGBRegressor(**best_parameters)

# Train the model
xgb_reg.fit(X_train, y_train,
            eval_set=evaluation, eval_metric="mape",
            early_stopping_rounds=20, verbose=False)

# Make predictions
pred = xgb_reg.predict(X_test)

# Calculate MAPE
mape = mean_absolute_percentage_error(y_test, pred)

print('MAPE :', mape)


MAPE : 11.0994085337936


In [42]:
xgb_reg.get_params

<bound method XGBModel.get_params of XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=0.9698294480528181, device=None,
             early_stopping_rounds=None, enable_categorical=False,
             eval_metric=None, feature_types=None, gamma=3.7604998424073166,
             grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.2718006286831415,
             max_bin=None, max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=8, max_leaves=None,
             min_child_weight=1, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=983, n_jobs=None,
             num_parallel_tree=None, random_state=123, ...)>

In [23]:
# Create the XGBRegressor model with the best parameters
xgb_reg2 = XGBRegressor(**xgb_optuna_params)

# Train the model
xgb_reg2.fit(X_train, y_train)

# Make predictions
pred = xgb_reg2.predict(X_test)

# Calculate MAPE
mape = mean_absolute_percentage_error(y_test, pred)

print('MAPE :', mape)

MAPE : 10.661403542422292


In [20]:
# Ensure that test dataset has the same columns and order as the training dataset
test = test[X_train.columns]

# Make a prediction based on the test dataset
submission = xgb_reg.predict(test)

# Create the submission dataframe
submission_df = pd.DataFrame(submission, index=test.index, columns=['Arrival_Delay'])

# Save the submission dataframe to a CSV file
submission_df.to_csv('submission_xgb.csv', index = False)


submission_df.head()


,Arrival_Delay
0,-7.746269
1,-2.656378
2,-2.143493
3,20.087856
4,157.024521


### Cross validation

In [24]:
%%time
# cross validation technique

from sklearn.model_selection import KFold
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_percentage_error

# Create the XGBRegressor model with the best parameters
xgb_reg1 = XGBRegressor(**best_parameters)


# Define the K-Fold cross-validation strategy
kfold = KFold(n_splits=7, shuffle=True, random_state=42)

# Initialize lists to store evaluation scores
mape_scores = []

# Perform K-Fold Cross-Validation
for train_index, test_index in kfold.split(X_train, y_train):
    X_train_fold, X_val_fold = X_train.iloc[train_index], X_train.iloc[test_index]
    y_train_fold, y_val_fold = y_train.iloc[train_index], y_train.iloc[test_index]

    # Fit the XGBoost model on the training fold
    xgb_reg1.fit(X_train_fold, y_train_fold)

    # Predict on the validation fold
    y_pred_val = xgb_reg1.predict(X_val_fold)

    # Calculate MAPE on the validation fold
    mape = mean_absolute_percentage_error(y_val_fold, y_pred_val)
    print('folds are working : ', mape)
    mape_scores.append(mape)

# Calculate mean MAPE across all folds
mean_mape = sum(mape_scores) / len(mape_scores)
print("Mean MAPE:", mean_mape)



folds are working :  6.366285706078069
folds are working :  8.56894356931949
folds are working :  5.305036953820558
folds are working :  7.559212543758011
folds are working :  16.537896080774853
folds are working :  7.2561559789939265
folds are working :  6.544750140861066
Mean MAPE: 8.30546871051514
CPU times: user 40min 53s, sys: 4.71 s, total: 40min 58s
Wall time: 10min 33s


In [25]:

# Make predictions
pred = xgb_reg1.predict(X_test)

# Calculate MAPE
mape = mean_absolute_percentage_error(y_test, pred)

print('MAPE :', mape)


MAPE : 12.745282548784498


### Apply feature selection method to reduce demention and prevent overfitting

In [26]:
import xgboost as xgb
from xgboost import XGBRegressor
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

# Assuming X_train and y_train are already defined
# Fit the model
xgb_model = XGBRegressor(**best_parameters)
xgb_model.fit(X_train, y_train)

# Get feature importances
importances = xgb_model.feature_importances_
feature_names = X_train.columns


In [29]:
# Create a dataframe for feature importances
importance_df = pd.DataFrame({'Feature': feature_names, 'Importance': importances})

# Sort the dataframe by importance
importance_df = importance_df.sort_values(by='Importance', ascending=False)

# Select the top N features (for example, top 10 features)
top_n = 10
top_features = importance_df.head(top_n)['Feature']

top_features

6                      Departure_Delay
20       Actual_Departure_Time_Minutes
18    Scheduled_Departure_Time_Minutes
7                        Taxi_Out_Time
8                         Taxi_In_Time
0                              Weekday
19      Scheduled_Arrival_Time_Minutes
10                     Flight_Duration
11                     Flight_Distance
3                      Departure_State
Name: Feature, dtype: object

In [30]:
# Reduce the dataset to the top N features
X_train_reduced = X_train[top_features]
X_test_reduced = X_test[top_features]

# Train the model again using only the selected features
xgb_reg_reduced = XGBRegressor(**best_parameters)
xgb_reg_reduced.fit(X_train_reduced, y_train)

# Make predictions on the test set
y_pred = xgb_reg_reduced.predict(X_test_reduced)

# Calculate MAPE
mape = mean_absolute_percentage_error(y_test, pred)

print('MAPE :', mape)



MAPE : 12.745282548784498


In [32]:
# select best features from test data
sub_test = test[top_features]
sub_test.shape, sub_test.columns

((407832, 10),
 Index(['Departure_Delay', 'Actual_Departure_Time_Minutes',
        'Scheduled_Departure_Time_Minutes', 'Taxi_Out_Time', 'Taxi_In_Time',
        'Weekday', 'Scheduled_Arrival_Time_Minutes', 'Flight_Duration',
        'Flight_Distance', 'Departure_State'],
       dtype='object'))

In [33]:
# Train the model again using only the selected features
xgb_reg_reduced = XGBRegressor(**best_parameters)
xgb_reg_reduced.fit(X_train_reduced, y_train)


# Make predictions on the test set
submission_reduced = xgb_reg_reduced.predict(sub_test)


# Create the submission dataframe
submission_df_reduced = pd.DataFrame(submission_reduced, index=test.index, columns=['Arrival_Delay'])

# Save the submission dataframe to a CSV file
submission_df_reduced.to_csv('submission_reduced.csv')

# Verify the file creation
submission_df_reduced.head()

,Arrival_Delay
0,-5.736109
1,-4.135200
2,-0.333087
3,20.312044
4,140.359421


# Voting regressor

In [51]:
lgb_params = {'boosting_type': 'gbdt',
 'class_weight': None,
 'colsample_bytree': 0.8083100557602823,
 'importance_type': 'split',
 'learning_rate': 0.5590257573381373,
 'max_depth': 8,
 'min_child_samples': 20,
 'min_child_weight': 6.329915255850705,
 'min_split_gain': 0.01593297353431914,
 'n_estimators': 1897,
 'n_jobs': -1,
 'num_leaves': 38,
 'objective': None,
 'random_state': None,
 'reg_alpha': 0.7801473610044566,
 'reg_lambda': 0.9168560003582829,
 'subsample': 0.560868988960792,
 'subsample_for_bin': 200000,
 'subsample_freq': 0,
 'nthread': 6.6516088270926605}


xgb_params = {'colsample_bytree': 0.9698294480528181,
 'gamma': 3.7604998424073166,
 'learning_rate': 0.2718006286831415,
 'max_depth': 8,
 'min_child_weight': 1,
 'n_estimators': 983,
 'reg_alpha': 146,
 'subsample': 0.7423469165690778,
 'random_state': 123,
 'n_jobs': -1}


cat_params = {'colsample_bylevel': 0.6312979928865734,
 'depth': 9,
 'l2_leaf_reg': 8.754132470161709,
 'learning_rate': 0.09396165335017637,
 'max_bin': 490.0,
 'random_strength': 0.07403229688797397,
 'subsample': 0.8801936829680834}


In [52]:
# Create individual regression models
xgb_reg = XGBRegressor(**xgb_params)
lgb_reg = LGBMRegressor(**lgb_params, silent=True)
cat_reg = CatBoostRegressor(**cat_params)

# create voting regressor
voting_reg = VotingRegressor(estimators=[
    ('lgb_reg', lgb_reg),
    ('xgb_reg', xgb_reg),
    ('cat_reg', cat_reg)
], n_jobs=-1)

voting_reg

VotingRegressor(estimators=[('lgb_reg',
                             LGBMRegressor(colsample_bytree=0.8083100557602823,
                                           learning_rate=0.5590257573381373,
                                           max_depth=8,
                                           min_child_weight=6.329915255850705,
                                           min_split_gain=0.01593297353431914,
                                           n_estimators=1897, n_jobs=-1,
                                           nthread=6.6516088270926605,
                                           num_leaves=38,
                                           reg_alpha=0.7801473610044566,
                                           reg_lambda=0.9168560003582829,
                                           silent=True,
                                           subsam...
                                          learning_rate=0.2718006286831415,
                                          max_bin=None, max_cat_threshold=None,
                                          max_cat_to_onehot=None,
                                          max_delta_step=None, max_depth=8,
                                          max_leaves=None, min_child_weight=1,
                                          missing=nan,
                                          monotone_constraints=None,
                                          multi_strategy=None, n_estimators=983,
                                          n_jobs=-1, num_parallel_tree=None,
                                          random_state=123, ...)),
                            ('cat_reg',
                             <catboost.core.CatBoostRegressor object at 0x78a4f720be20>)],
                n_jobs=-1)

In [53]:
# Fit the Voting Regressor on the training data
voting_reg.fit(X_train, y_train)

0:	learn: 52.2988360	total: 630ms	remaining: 10m 29s
1:	learn: 48.5442348	total: 1.09s	remaining: 9m 4s
2:	learn: 45.2010356	total: 1.52s	remaining: 8m 24s
3:	learn: 42.2420585	total: 1.94s	remaining: 8m 2s
4:	learn: 39.6348820	total: 2.38s	remaining: 7m 54s
5:	learn: 37.2905140	total: 2.83s	remaining: 7m 48s
6:	learn: 35.2658472	total: 3.3s	remaining: 7m 47s
7:	learn: 33.3809861	total: 3.74s	remaining: 7m 43s
8:	learn: 31.7522412	total: 4.22s	remaining: 7m 44s
9:	learn: 30.2763986	total: 4.68s	remaining: 7m 43s
10:	learn: 29.0260274	total: 5.1s	remaining: 7m 38s
11:	learn: 27.9137752	total: 5.52s	remaining: 7m 34s
12:	learn: 26.8954582	total: 6s	remaining: 7m 35s
13:	learn: 26.0388788	total: 6.47s	remaining: 7m 35s
14:	learn: 25.3023984	total: 6.9s	remaining: 7m 33s
15:	learn: 24.5342608	total: 7.42s	remaining: 7m 36s
16:	learn: 23.9504047	total: 7.88s	remaining: 7m 35s
17:	learn: 23.4128649	total: 8.34s	remaining: 7m 34s
18:	learn: 22.9385407	total: 8.8s	remaining: 7m 34s
19:	learn: 

VotingRegressor(estimators=[('lgb_reg',
                             LGBMRegressor(colsample_bytree=0.8083100557602823,
                                           learning_rate=0.5590257573381373,
                                           max_depth=8,
                                           min_child_weight=6.329915255850705,
                                           min_split_gain=0.01593297353431914,
                                           n_estimators=1897, n_jobs=-1,
                                           nthread=6.6516088270926605,
                                           num_leaves=38,
                                           reg_alpha=0.7801473610044566,
                                           reg_lambda=0.9168560003582829,
                                           silent=True,
                                           subsam...
                                          learning_rate=0.2718006286831415,
                                          max_bin=None, max_cat_threshold=None,
                                          max_cat_to_onehot=None,
                                          max_delta_step=None, max_depth=8,
                                          max_leaves=None, min_child_weight=1,
                                          missing=nan,
                                          monotone_constraints=None,
                                          multi_strategy=None, n_estimators=983,
                                          n_jobs=-1, num_parallel_tree=None,
                                          random_state=123, ...)),
                            ('cat_reg',
                             <catboost.core.CatBoostRegressor object at 0x78a4f720be20>)],
                n_jobs=-1)

In [54]:
# Make predictions
pred = voting_reg.predict(X_test)

# Calculate MAPE
mape = mean_absolute_percentage_error(y_test, pred)

print('MAPE :', mape)


[LightGBM] [Warning] Unknown parameter: silent
MAPE : 10.377748623898487
[LightGBM] [Warning] Unknown parameter: silent
[LightGBM] [Warning] Unknown parameter: silent
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.344335 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2954
[LightGBM] [Info] Number of data points in the train set: 1065015, number of used features: 19
[LightGBM] [Info] Start training from score 7.538319


In [56]:
print(mape)

10.377748623898487


### Use reduced features

In [57]:
# Reduce the dataset to the top N features
X_train_reduced = X_train[top_features]
X_test_reduced = X_test[top_features]

# Create individual regression models
xgb_reg = XGBRegressor(**xgb_params)
lgb_reg = LGBMRegressor(**lgb_params, silent=True)
cat_reg = CatBoostRegressor(**cat_params)

# create voting regressor
voting_reg1 = VotingRegressor(estimators=[
    ('lgb_reg', lgb_reg),
    ('xgb_reg', xgb_reg),
    ('cat_reg', cat_reg)
], n_jobs=-1)


voting_reg1.fit(X_train_reduced, y_train)



[LightGBM] [Warning] Unknown parameter: silent
[LightGBM] [Warning] Unknown parameter: silent
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.199793 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1832
[LightGBM] [Info] Number of data points in the train set: 1065015, number of used features: 10
[LightGBM] [Info] Start training from score 7.538319
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
0:	learn: 52.0063732	total: 228ms	remaining: 3m 47s
1:	learn: 48.3043614	total: 392ms	remaining: 3m 15s
2:	learn: 45.0177691	total: 559ms	remaining: 3m 5s
3:	learn: 42.1504175	total: 722ms	remaining: 2m 59s
4:	learn: 39.5417754	total: 886ms	remaining: 2m 56s
5:	learn: 37.2494719	total: 1.05s	remaining: 2m 54s
6:	learn: 35.1427082	total: 1.22s	remaining: 2m 52s
7:	learn: 33.3097206	total: 1.38s	remaining: 2m 5

VotingRegressor(estimators=[('lgb_reg',
                             LGBMRegressor(colsample_bytree=0.8083100557602823,
                                           learning_rate=0.5590257573381373,
                                           max_depth=8,
                                           min_child_weight=6.329915255850705,
                                           min_split_gain=0.01593297353431914,
                                           n_estimators=1897, n_jobs=-1,
                                           nthread=6.6516088270926605,
                                           num_leaves=38,
                                           reg_alpha=0.7801473610044566,
                                           reg_lambda=0.9168560003582829,
                                           silent=True,
                                           subsam...
                                          learning_rate=0.2718006286831415,
                                          max_bin=None, max_cat_threshold=None,
                                          max_cat_to_onehot=None,
                                          max_delta_step=None, max_depth=8,
                                          max_leaves=None, min_child_weight=1,
                                          missing=nan,
                                          monotone_constraints=None,
                                          multi_strategy=None, n_estimators=983,
                                          n_jobs=-1, num_parallel_tree=None,
                                          random_state=123, ...)),
                            ('cat_reg',
                             <catboost.core.CatBoostRegressor object at 0x78a4f73461d0>)],
                n_jobs=-1)

In [58]:
# Make predictions on the test set
y_pred = voting_reg1.predict(X_test_reduced)

# Calculate MAPE
mape = mean_absolute_percentage_error(y_test, pred)

print('MAPE :', mape)


[LightGBM] [Warning] Unknown parameter: silent
MAPE : 10.377748623898487


### Submission with voting

In [59]:
# select best features from test data
sub_test = test[top_features]

# Make predictions on the test set
submission_reduced = voting_reg1.predict(sub_test)


# Create the submission dataframe
submission_df_reduced = pd.DataFrame(submission_reduced, index=test.index, columns=['Arrival_Delay'])



[LightGBM] [Warning] Unknown parameter: silent


,Arrival_Delay
0,-5.501700
1,-0.380695
2,-1.648957
3,19.215405
4,75.175441


In [62]:
# Save the submission dataframe to a CSV file
submission_df_reduced.to_csv('submission_reduced_voting.csv', index = False)

# Verify the file creation
submission_df_reduced.head()

,Arrival_Delay
0,-5.501700
1,-0.380695
2,-1.648957
3,19.215405
4,75.175441


In [63]:
voting_sub = pd.read_csv('/kaggle/working/submission_reduced_voting.csv')

voting_sub.head()

,Arrival_Delay
0,-5.501700
1,-0.380695
2,-1.648957
3,19.215405
4,75.175441


In [66]:
xgb_sub = pd.read_csv('/kaggle/working/submission_reduced.csv')
xgb_sub['Arrival_Delay'].to_csv('submission_reduced.csv', index = False)


xgb_sub = pd.read_csv('/kaggle/working/submission_reduced.csv')
xgb_sub.head()

,Arrival_Delay
0,-5.736109
1,-4.135199
2,-0.333087
3,20.312044
4,140.359420


In [67]:
# Assuming the predictions are in a column named 'prediction'
absolute_errors = abs(xgb_sub['Arrival_Delay'] - voting_sub['Arrival_Delay'])

# Calculate Mean Absolute Difference (MAD) as a proxy for RAE
mean_absolute_difference = absolute_errors.mean()

print("Mean Absolute Difference between the two models:", mean_absolute_difference)

Mean Absolute Difference between the two models: 10.394640606543627
